# Project1

## Step 1
Dependencies and Setup

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json

from scipy.stats import linregress

from api_keys import geoapify_key

In [2]:
# Load the CSV file
df=pd.read_csv("./Resources/Kaggle/Student Loan Debt by School 2020-2021.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42740 entries, 0 to 42739
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  42740 non-null  int64  
 1   OPE ID                 42740 non-null  int64  
 2   School                 42740 non-null  object 
 3   City                   33650 non-null  object 
 4   State                  40700 non-null  object 
 5   Zip Code               33650 non-null  float64
 6   School Type            42740 non-null  object 
 7   Loan Type              42740 non-null  object 
 8   Recipients             42740 non-null  float64
 9   # of Loans Originated  42740 non-null  float64
 10  $ of Loans Originated  42740 non-null  float64
 11  # of Disbursements     42740 non-null  float64
 12  $ of Disbursements     42740 non-null  float64
dtypes: float64(6), int64(2), object(5)
memory usage: 4.2+ MB


In [3]:
# Change data type and rename columns
df['Zip Code'] = df['Zip Code'].fillna(-1).astype(int).astype(str).replace('-1', 'NaN')
df['Recipients'] = df['Recipients'].astype(int)
df['# of Loans Originated'] = df['# of Loans Originated'].astype(int)
df['# of Disbursements'] = df['# of Disbursements'].astype(int)
df=df[df['Loan Type']!='Total']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21370 entries, 0 to 21369
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  21370 non-null  int64  
 1   OPE ID                 21370 non-null  int64  
 2   School                 21370 non-null  object 
 3   City                   16825 non-null  object 
 4   State                  20350 non-null  object 
 5   Zip Code               21370 non-null  object 
 6   School Type            21370 non-null  object 
 7   Loan Type              21370 non-null  object 
 8   Recipients             21370 non-null  int32  
 9   # of Loans Originated  21370 non-null  int32  
 10  $ of Loans Originated  21370 non-null  float64
 11  # of Disbursements     21370 non-null  int32  
 12  $ of Disbursements     21370 non-null  float64
dtypes: float64(2), int32(3), int64(2), object(6)
memory usage: 2.0+ MB


In [4]:
# df.head(10)
df.tail(10)

,index,OPE ID,School,City,State,Zip Code,School Type,Loan Type,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements
21360,21360,42919,FLORIDA PROFESSIONAL INSTITUTE,MIAMI,FL,33155,Proprietary,Grad Plus,0,0,0.0,0,0.0
21361,21361,34165,DALLAS NURSING INSTITUTE,NaN,TX,NaN,Private-Nonprofit,Grad Plus,0,0,0.0,0,0.0
21362,21362,21265,ENID BEAUTY COLLEGE,NaN,OK,NaN,Proprietary,Grad Plus,0,0,0.0,0,0.0
21363,21363,42907,PREMIERE AESTHETICS INSTITUTE,BEND,OR,97702,Proprietary,Grad Plus,0,0,0.0,0,0.0
21364,21364,41869,"INTERNATIONAL COLLEGE OF BEAUTY, ARTS & SCIENCES",LOS ANGELES,CA,90039,Proprietary,Grad Plus,0,0,0.0,0,0.0
21365,21365,42377,REGIONAL CENTER FOR BORDER HEALTH,SOMERTON,AZ,85350,Private-Nonprofit,Grad Plus,0,0,0.0,0,0.0
21366,21366,40293,TRENDSETTERS OF FL. SCHOOL OF BEAUTY & BARBERING,JACKSONVILLE,FL,32205,Proprietary,Grad Plus,0,0,0.0,0,0.0
21367,21367,4666,AMERICAN COLLEGE FOR MEDICAL CAREERS,ORLANDO,FL,32809,Proprietary,Grad Plus,0,0,0.0,0,0.0
21368,21368,42932,PROFESSIONAL COSMETOLOGY ACADEMY (THE),LAREDO,TX,78041,Proprietary,Grad Plus,0,0,0.0,0,0.0
21369,21369,42185,INNOVATE SALON ACADEMY,SOUTH PLAINFIELD,NJ,7080,Proprietary,Grad Plus,0,0,0.0,0,0.0


In [5]:
df.describe()

,index,OPE ID,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements
count,21370.000000,21370.000000,21370.000000,21370.000000,2.137000e+04,21370.000000,2.137000e+04
mean,10684.500000,16201.447824,327.571221,336.351006,2.552457e+06,341.877679,1.278388e+06
std,6169.131962,15479.675287,1026.847813,1057.889512,8.464158e+06,1078.516745,4.298032e+06
min,0.000000,1002.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,5342.250000,2848.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
50%,10684.500000,8447.500000,10.000000,10.000000,5.841800e+04,10.000000,2.873850e+04
75%,16026.750000,30837.000000,188.750000,192.000000,1.430073e+06,197.000000,7.090422e+05
max,21369.000000,42933.000000,26663.000000,27261.000000,2.586015e+08,27261.000000,1.085654e+08


## Step 2
### Ask the questions

State, What is the breakdown of number of collages in each state? P

State, What is the breakdown of recipients of Student Loans by State (by quantity and by $)?P

State, What states have the most (public schools? Private schools?) recipients?P

State, What are the top 10 states with the most student loans?L

Geographic, How does the average student loan vary by zip code?M

Geographic, Are there trends based on geographic region or type of school?T

In [6]:
# get US, none zero data
drop_df= df[(df['Zip Code']!='NaN')&(df['Zip Code']!='0')&(df['Recipients']!=0)]

# drop_df.to_csv('./Output/test.csv')
drop_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11471 entries, 0 to 21252
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  11471 non-null  int64  
 1   OPE ID                 11471 non-null  int64  
 2   School                 11471 non-null  object 
 3   City                   11471 non-null  object 
 4   State                  11471 non-null  object 
 5   Zip Code               11471 non-null  object 
 6   School Type            11471 non-null  object 
 7   Loan Type              11471 non-null  object 
 8   Recipients             11471 non-null  int32  
 9   # of Loans Originated  11471 non-null  int32  
 10  $ of Loans Originated  11471 non-null  float64
 11  # of Disbursements     11471 non-null  int32  
 12  $ of Disbursements     11471 non-null  float64
dtypes: float64(2), int32(3), int64(2), object(6)
memory usage: 1.1+ MB


In [9]:
# # create zipwithLL.csv
# zip_df=pd.read_csv("./Resources/Zipcode/uszips.csv")
# zip_df['Full Zip Code']= zip_df['Zip Code'].astype(str).str.zfill(5)
# zip_df['Zip Code']=zip_df['Zip Code'].astype(str)
# zip_df.tail(10)


# base_url='https://api.geoapify.com/v1/geocode/search?'
# params = {
    
#     "apiKey": geoapify_key,
#     "type":"postcode",
#     'format':'json',
#     'country':'USA'
# }


# for index, row in zip_df.iterrows():

#     params["text"] = row['Full Zip Code']
#     params["city"] = row['City']
#     params["state"] = row['State']
#     if pd.isnull(row['Lat']):

#         LL = requests.get(base_url, params=params)

#     # Convert the API response to JSON format
#         LL = LL.json()
            
#         try:
#             # print(row['Full Zip Code'])         
#             # print(LL["results"][0]["lat"])
#             # print(LL["results"][0]["lon"])
#             zip_df.loc[index, "Lat"] = LL["results"][0]["lat"]
#             zip_df.loc[index, "Lng"] = LL["results"][0]["lon"]
#         except (KeyError, IndexError):
#             params.pop("text")
#             params.pop("type")
#             LL = requests.get(base_url, params=params)
#             # print(params)
#             LL = LL.json()
#             # print(json.dumps(LL,indent=4))
#             zip_df.loc[index, "Lat"] = LL["results"][0]["lat"]
#             zip_df.loc[index, "Lng"] = LL["results"][0]["lon"]
# zip_df.to_csv('./Output/zipwithLL.csv',index=False)


In [12]:
zip_df=pd.read_csv("./Output/zipwithLL.csv")
zip_df['Full Zip Code']= zip_df['Zip Code'].astype(str).str.zfill(5)
zip_df['Zip Code']=zip_df['Zip Code'].astype(str)
drop_LL_df=pd.merge(drop_df,zip_df,how="left",on='Zip Code')

In [14]:
drop_LL_df


,index,OPE ID,School,City_x,State_x,Zip Code,School Type,Loan Type,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements,Lat,Lng,City_y,State_y,Full Zip Code
0,0,1051,UNIVERSITY OF ALABAMA,TUSCALOOSA,AL,35487,Public,Subsidized,7594,7622,33040862.0,7622,16774054.0,33.214260,-87.547820,Tuscaloosa,AL,35487
1,1,1052,UNIVERSITY OF ALABAMA AT BIRMINGHAM,BIRMINGHAM,AL,35294,Public,Subsidized,4272,4352,18590061.0,4352,9058344.0,33.520682,-86.802433,BIRMINGHAM,AL,35294
2,2,1009,AUBURN UNIVERSITY,AUBURN,AL,36849,Public,Subsidized,4504,4504,19474542.0,4535,10142490.0,32.599530,-85.488580,Auburn University,AL,36849
3,3,1057,UNIVERSITY OF SOUTH ALABAMA,MOBILE,AL,36688,Public,Subsidized,3438,3487,15061071.0,3487,7557853.0,30.695590,-88.187830,Mobile,AL,36688
4,4,1047,TROY UNIVERSITY,TROY,AL,36082,Public,Subsidized,4334,4355,18467508.0,4355,7395839.0,31.802710,-85.955140,Troy,AL,36082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11466,20998,2536,UNIVERSITY OF MONTANA (THE),MISSOULA,MT,59812,Public,Grad Plus,352,367,4975144.0,367,2519341.0,46.861210,-113.984420,Missoula,MT,59812
11467,21017,41944,AMERICAN COLLEGE OF HEALTHCARE SCIENCES,PORTLAND,OR,97239,Proprietary,Grad Plus,2,2,20494.0,2,10247.0,45.488930,-122.690530,Portland,OR,97239
11468,21217,2532,MONTANA STATE UNIVERSITY BOZEMAN,BOZEMAN,MT,59717,Public,Grad Plus,84,86,818532.0,86,408021.0,45.679429,-111.044047,BOZEMAN,MT,59717
11469,21236,42542,INTER AMERICAN UNIVERSITY OF PUERTO RICO SAN G...,SAN GERMAN,PR,683,Private-Nonprofit,Grad Plus,1,1,18000.0,1,9000.0,18.109260,-67.038030,San German,PR,00683


Loan Type, What is the breakdown of type of student loan  (subsidized, unsubsidized, grad plus, parent plus, etc.)?M

School Type, What is the breakdown of student loan quantity/$ by school type (for profit, non-profit, public, etc.)?L

<!-- VS, What is the breakdown in disbursements versus loan origination $ amounts? -->

<!-- VS, Which schools have the highest disparity between loans originated and loans disbursed? -->

API, Foreign university, VS US

More datasets